In [8]:
import pandas as pd
import numpy as np
import torch

contentDataframe = pd.read_excel('Data/Online Retail.xlsx')
contentDataframe.dropna(subset=['CustomerID','InvoiceDate'])
contentDataframe = contentDataframe.sort_values(by=["CustomerID", 'InvoiceDate'],kind="mergesort") #stable sorting means I can sort by one column instead of two :)


In [7]:
display(contentDataframe)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom
14938,537626,85116,BLACK CANDELABRA T-LIGHT HOLDER,12,2010-12-07 14:57:00,2.10,12347.0,Iceland
14939,537626,22375,AIRLINE BAG VINTAGE JET SET BROWN,4,2010-12-07 14:57:00,4.25,12347.0,Iceland
14940,537626,71477,COLOUR GLASS. STAR T-LIGHT HOLDER,12,2010-12-07 14:57:00,3.25,12347.0,Iceland
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


In [ ]:

contentDataframe['InvoiceDate'] = pd.to_datetime(contentDataframe['InvoiceDate'])
contentDataframe['Date'] = contentDataframe['InvoiceDate'].dt.date
contentDataframe['Revenue'] = contentDataframe['Quantity'] * contentDataframe['UnitPrice']

dailyDataframe = contentDataframe.groupby('Date').agg({'Revenue': 'sum'}).reset_index()
allDates = pd.date_range(start=dailyDataframe['Date'].min(), end=dailyDataframe['Date'].max())
allCustomers = dailyDataframe['CustomerID'].unique()

allTimes = pd.MultiIndex.from_product([allCustomers, allDates], names=['CustomerID', 'Date'])
dailyDataframe = dailyDataframe.set_index(['CustomerID', 'Date']).reindex(allTimes).reset_index()

dailyDataframe['Quantity'] = dailyDataframe['Quantity'].fillna(0)
dailyDataframe['Revenue'] = dailyDataframe['Revenue'].fillna(0)

dailyDataframe['time_idx'] = dailyDataframe.groupby('CustomerID').cumcount()

In [ ]:
validationRows = []
testingRows = []
for CustomerID, group in dailyDataframe.groupby('CustomerID'):
    numberOfRows = len(group)
    last20 = int(np.floor(0.2*numberOfRows))
    last20Rows = group.tail(last20)
    validationSplit = group.head(last20 / 2)
    testingSplit = group.tail(last20 / 2)
    validationRows.append(validationSplit)
    testingRows.append(testingSplit)
validationDataframe = pd.concat(validationRows, ignore_index=True)
testingDataframe = pd.concat(testingRows,ignore_index=True)

In [ ]:
training = TimeSeries